In [3]:
import csv
import matplotlib
import pandas as pd
import json
import os
import math
import re
from utils.postProcessing import *#img_ellipse_fitting
from utils.imageUtils import cropImage
from utils import imageUtils, postProcessing, DefectDetectionDataset, evaluation, visualization
import matplotlib.pyplot as plt

In [4]:
imagefolder = '/u/y/u/yuhanl/Downloads/NextGenMaskRCNN-master/code/datasets/balloon/Data3TypesYminXminYmaxXmax5/images/'
root = '/u/y/u/yuhanl/Downloads/NextGenMaskRCNN-master/code/datasets/balloon/Data3TypesYminXminYmaxXmax5'

# files = os.listdir('/srv/home/whao/NextGen/new_debug/run/run5/datasets/balloon/train/')    
dataset = DefectDetectionDataset(data_dir = root, split = 'train')

All_Image_Defects_List = list()
for item in range(len(dataset)):
    #print(item)
    img, bboxes, label = dataset[item]
    subimages, bboxes = cropImage(img, bboxes)
    current_img_defect_List = list()
    for subim, bbox in zip(subimages, bboxes):
        defects_Dict = dict()
        defects_X_List = list()
        defects_Y_List = list()
        # use watershed algorithms to fit the ellipse using the data in the bbox
        region1 = flood_fitting(subim,bbox[0])
        
        result = (int(region1['centroid'][0]+bbox[1]), int(region1['centroid'][1]+bbox[2]),
                    int(region1['minor_axis_length'] / 2), int(region1['major_axis_length'] / 2),
                    -region1['orientation'])
        rr,cc = draw.ellipse_perimeter(*result)

        assert len(rr) == len(cc)
        for i in range(len(cc)):
            defects_X_List.append(cc[i])
            defects_Y_List.append(rr[i])
        defects_Dict['X'] = defects_X_List
        defects_Dict['Y'] = defects_Y_List
        current_img_defect_List.append(defects_Dict)
    All_Image_Defects_List.append(current_img_defect_List)

/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:25

/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:25

/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


In [ ]:
len(All_Image_Defects_List)

In [ ]:
def int64Toint32(list):
    for item in list:
        #print( type(item))
        item = int(item)
        #print( type(item) )
    return list

In [5]:
with open("via_region_data.json","w+") as jsonfile:
    with open("/u/y/u/yuhanl/Downloads/NextGenMaskRCNN-master/code/datasets/balloon/Data3TypesYminXminYmaxXmax5/trainimages.txt") as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    each_file = dict()
    for i in range(len(All_Image_Defects_List)):
        img_i = All_Image_Defects_List[i]
        file_size = os.path.getsize(imagefolder + '/'+content[i])
        filename_size = content[i] + str(file_size)
        each_file[filename_size] = {'fileref':"",'size':file_size,'filename':content[i]}
        each_file[filename_size].update({'base64_img_data':"",'file_attributes':{},'regions':{}})
        each_bbox = each_file[filename_size]['regions']
        for j in range(len(img_i)):
            each_bbox.update({str(j):{'shape_attributes':{},'region_attributes':{}}})
            each_bbox[str(j)]['shape_attributes'] = {'name':"polygon"}
            each_bbox[str(j)]['shape_attributes'].update({'all_points_x':[],'all_points_y':[]})
            each_bbox[str(j)]['shape_attributes']['all_points_x'] = np.asarray(img_i[j]['X']).tolist()#int64Toint32(img_i[j]['X'])
            each_bbox[str(j)]['shape_attributes']['all_points_y'] = np.asarray(img_i[j]['Y']).tolist()#int64Toint32(img_i[j]['Y'])

    json.dump(each_file,jsonfile)